# ENOT Lite Framework: Getting Started

This is a minimal example of using ENOT Lite Framework.

In [ ]:
# ENOT Lite Framework does not depends on these packages.
# They are used for illustrative purposes.

import inspect
import urllib.request
import numpy as np
from PIL import Image
from PIL import ImageDraw
from torchvision import transforms

To import all available backends simply import `backend` module from `enot_lite`:

In [ ]:
from enot_lite import backend

Now you can inspect available backends by the following code:

In [ ]:
for back, _ in inspect.getmembers(backend, inspect.isclass):
    print(back)

Let's try using `OrtTensorrtAutoBackend` backend for task of image classification.

### Model, labels and image
At first we should download model, labels and image for classification:

In [ ]:
urllib.request.urlretrieve(url='https://github.com/onnx/models/raw/main/vision/classification/squeezenet/model/squeezenet1.1-7.onnx', filename='squeezenet1.1-7.onnx');
urllib.request.urlretrieve(url='https://s3.amazonaws.com/onnx-model-zoo/synset.txt', filename='synset.txt');
urllib.request.urlretrieve(url='https://s3.amazonaws.com/model-server/inputs/kitten.jpg', filename='kitten.jpg');

Open and prepare image that will be classified:

In [ ]:
def preprocess(img):
    transform_fn = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )
    img = transform_fn(img)
    img = img.unsqueeze(0)
    return img.numpy()

image = Image.open('kitten.jpg')
image = preprocess(image)

Get labels from downloaded file:

In [ ]:
with open('synset.txt', 'r') as file:
    labels = [line.rstrip() for line in file]

### Backend usage

To construct new backend you need to provide model and (optionally) session options.

In [ ]:
sess = backend.OrtTensorrtAutoBackend('squeezenet1.1-7.onnx')

Run inference to get prediction by using `run` method of our backend:

In [ ]:
scores = sess.run(image)[0]

### Visualize prediction

In [ ]:
top_class = labels[np.argsort(np.squeeze(scores))[::-1][0]]

In [ ]:
classified = Image.open('kitten.jpg')
classified = classified.resize((256, 256))
draw = ImageDraw.Draw(classified)
draw.text((60, 10), top_class, (255, 255, 255))
display(classified)